In [ ]:
import pandas as pd

# 파일 경로 설정, 확장자가 .cs
file_path = 'D:/시설/정지/일일정지C150_G0000_00008.csv'

# CSV 파일 읽기, 헤더 및 데이터 타입 경고 처리
df = pd.read_csv(file_path, encoding='cp949', low_memory=False)

# 헤더만 출력, 보기 좋게 줄바꿈하여 표시
for column in df.columns:
    print(column)

In [ ]:
import pandas as pd

# 데이터 로드 및 초기 처리 예시
# df = pd.read_csv('file_path.csv', encoding='cp949')
# df.fillna('', inplace=True)  # NaN 값을 빈 문자열로 대체

# 데이터 타입 변환
df['계약번호'] = pd.to_numeric(df['계약번호'], errors='coerce')
df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'], errors='coerce')



# 본부별 지사 매핑
본부_지정 = {
    '강남/서부본부': ['강남지사', '강동지사', '강서지사', '관악지사', '부천지사', '분당지사', '수원지사', '안산지사', '안양지사', '용인지사', '인천지사', '평택지사'],
    '강북/강원본부': ['중앙지사', '강북지사', '고양지사', '서대문지사', '의정부지사', '남양주지사', '원주지사', '춘천고객지원팀',  '강릉지사'],
    '대구/경북본부': ['대구지사', '동대구지사', '수성지사', '구미지사', '포항지사'],
    '부산/경남본부': ['동부산지사', '남부산지사', '서부산지사', '울산지사', '김해지사', '창원지사', '진주지사'],
    '전남/전북본부': ['광주지사', '북광주지사', '목포지사', '순천지사', '전주지사', '익산지사', '제주지사'],
    '충남/충북본부': ['대전지사', '서대전지사', '천안지사', '충남서부지사', '충북지사']
}

# 본부별 DataFrame 생성
dataframes = {}
for 본부, 지사리스트 in 본부_지정.items():
    본부_df = df[df['관리지사명'].isin(지사리스트)]
    본부_df_grouped = 본부_df.groupby('관리지사명').agg(
        계약번호_행개수=('계약번호', 'size'),
        KTT월정료_합계=('KTT월정료(조정)', 'sum')
    ).reset_index()
    본부_df_grouped['KTT월정료_합계'] = 본부_df_grouped['KTT월정료_합계'].apply(lambda x: f"{x:,.0f}")

    # 소계 추가
    소계 = 본부_df_grouped.sum(numeric_only=True)
    소계['관리지사명'] = '소계'
    소계_df = pd.DataFrame([소계], columns=본부_df_grouped.columns)
    본부_df_final = pd.concat([본부_df_grouped, 소계_df], ignore_index=True)

    dataframes[본부] = 본부_df_final

# 스타일링 함수 정의
def style_specific(df):
    return df.style.set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#f4f4f4'), ('color', 'black')]},
        {'selector': 'td', 'props': [('border-color', 'lightgray')]}
    ]).set_properties(**{
        'text-align': 'left',
        'border-color': 'lightgray',
        'border-width': '1px',
        'border-style': 'solid'
    })

# 본부별로 출력 (예시)
for 본부, 본부_df in dataframes.items():
    print(f"--- {본부} ---")
    display(style_specific(본부_df))

In [ ]:
ㅠㅠㅠ

In [ ]:
# 주소, 날짜형식 지정(본사용)

import pandas as pd

# 파일 경로 설정 (확장자 .cs)
file_path = 'D:/시설/유지/24년12월유지가입자.cs'

# 필요한 열만 지정하여 파일 읽기
usecols = [
    '관리본부명', '관리지사명', '계약번호', '서비스(대)', '서비스(중)', '서비스(소)', '상호',
    '고객구분', '사업용구분', '주민등록번호', '사업자번호', '계약상태(중)', '서비스상태(중)',
    '설치주소', '월정료', '계약개월수', '계약시작일', '계약종료일', '영업구분', '추천구분',
    '영업자명', '영업자연락처', '모집유형', '영업채널', '유통망대분류', '유통망중분류', '유통망소분류',
    '추천자명', '추천채널', '정보제공자명', '청구본부명', '청구지사명', '청구자명', '합산여부',
    '수납구분', '선후납구분', '청구번호', '청구주소', '준공완료일', '관제개통일', '서비스개시일',
    '서비스재개시일', '정지시작일자', '종목', '업태', '업종_대', '업종_중', '보험등급', '비고',
    '결합구분', '결합시작일', 'E-MAIL', '담당자', '휴대폰', '계약최초서비스게시일', '매출본사',
    '매출인정구분', '구역정보', '시설구분', '요금구분', '고객계약유형', '실적본부', '실적지사',
    '실적최초등록일', '실적구분', '영업구역정보', '기술구역정보', 'KT마스터ID', 'BIZNARU계약ID',
    '신규PP', 'SW주장치안정화DB', '매출구분', '영업본부2', '영업지사2', '실적채널', '실적부서',
    '직접/추천', '무인매장구분', '무인매장업종구분', '판매구분', 'KTT월정료(조정)', 'KT월정료(조정)',
    '고알프', '제외사유'
]

# 파일 읽기
df = pd.read_csv(file_path, encoding='cp949', usecols=usecols, low_memory=False)

# 날짜 변환 대상 열
date_columns = [
    '계약시작일', '계약종료일', '준공완료일', '관제개통일', '서비스개시일', '서비스재개시일',
    '정지시작일자', '결합시작일', '계약최초서비스게시일', '실적최초등록일'
]

# 날짜 열 변환 (yyyy-mm-dd)
for col in date_columns:
    if col in df.columns:  # 열이 존재하는 경우에만 처리
        df[col] = pd.to_datetime(df[col], format='%Y%m%d', errors='coerce').dt.strftime('%Y-%m-%d')

# "관리본부명" 조건에 맞는 데이터 필터링
filtered_data = df[df['관리본부명'].isin(['강북/강원본부', '강원본부'])]

# "관리본부명" 값 변경: "강원본부"를 "강북강원본부"로 변경
filtered_data['관리본부명'] = filtered_data['관리본부명'].replace('강원본부', '강북/강원본부')

# 필터링된 데이터 저장 (엑셀)
output_path = 'D:/시설/유지/24년12월전체유지조건요약_주소db.xlsx'
filtered_data.to_excel(output_path, index=False, engine='openpyxl')  # 엑셀 엔진 지정

# 결과 확인
print("조건에 맞는 데이터가 저장되었습니다:", output_path)